# Simplified Scripts with Google's Python Fire

- Why Fire
- Harnessing Fire
- Testing Fire
- Packaging Fire
- Distributing Fire
- Fire prevention
- Fire alternatives

# Why Fire

Fire dries-out argument parsing, routing, and help documentation by recursively inspecting objects' method signatures and docstrings then exposing these to the command line.

- automates CLI creation from ANY Python object through reflection
- rapid prototying of reusable components
- group, nest, and pipeline commands using OOP concepts & patterns
- interactive debugging with preloaded REPL
- exploring existing codebases
- transitioning from Bash to Python

# Why Fire

Fire **automates CLI creation**. If called without its component argument, Fire generates a command from every globally scoped identifer. Generated commands include positional and keyword arguments without additional declarations.

In [1]:
import fire

def greet(name, msg='Hello {}, welcome to Automate!'):
    """Greet people with templated message."""
    return msg.format(name)

def main():
    fire.Fire()

In [2]:
%%bash
python ./demo/rad.py greet everyone
python ./demo/rad.py greet pythonistas --msg='Hello {}, thank you for coming!'

Hello everyone, welcome to Automate!
Hello pythonistas, thank you for coming!


# Why Fire

Fire simplifies **rapid prototying of reusable components**. Just refactor components as needed, then factor out components as needed; there's nothing impeding you.

In [3]:
import fire
from translate import Translator

def greet(name, msg='Hello {}, welcome to Automate!', lang='en'):
    """Greet people in their preferred language."""
    return Translator(lang).translate(msg.format(name))

def main():
    fire.Fire()

In [4]:
%%bash
python ./demo/refactored.py greet everyone --lang=hi

सभी को नमस्कार, स्वचालित में आपका स्वागत है!


# Why Fire

Fire **simplifies grouping, nesting, chaining, and pipelining** commands through OOP.

In [5]:
class NestedGroup(object):

    def command(self):
        print('Fired NestedGroup command!')


class ParentGroup(object):
    nested = NestedGroup()

    def command(self):
        print('Fired ParentGroup command')

def main():
    fire.Fire(ParentGroup)

In [6]:
%%bash
python ./demo/routing_advanced.py command
python ./demo/routing_advanced.py nested command

Fired ParentGroup's command method!
Fired NestedGroup's command method!


- grouping accomplished with classes
- nesting & pipelining accomplished with composition & delgation
- chaining accompolished with Method Channing Design Pattern

# Why Fire

Fire provides **interactive debugging with preloaded REPL** & optional IPython REPL. 

In [7]:
%%bash
# python ./demo/repl.py -- --interactive

<img src='./images/repl.png' />

- IPython dependency removed May 2019: commeit 952e20d1d5e0c264dd17f4f52ebf28f6a194b4c1

# Why Fire

Fire **eases transiting from Bash to Python** by handling stdin and stdout for you.

In [9]:
import fire
from translate import Translator

GREETING = 'Hello {}, welcome to IndyPy!'

def greet(name, language='en', msg=GREETING):
    """Greet people in their preferred language."""
    return Translator(language).translate(msg.format(name))

def main():
    fire.Fire()

In [10]:
%%bash
python ./demo/rad.py greet everyone
echo "greet everyone" | xargs python ./demo/rad.py | wc

Hello everyone, welcome to Automate!
       1       5      37


# Harnessing Fire
1. Installation
1. Implicit & Explicit routing
1. Commands
1. Grouped and nested routes
1. Piped 
1. Parameter handling

# Harnessing Fire: Installation

- requires Python 2.7, 3.4, 3.5, or 3.6
- haven't experienced issues on 3.7, but not recommend

__pipenv__
```bash
pipenv install fire                  # if python 3.6 first in $PATH
pipenv install fire --python 3.6     # if pyenv installed & configured
```

__miniconda with pip-tools__
```bash
conda create python=3.6 --name <env>
conda activate <env>
conda install fire -c conda-forge
echo fire >> requirements.in         # pip-tools replacement yet?
```

# Harnessing Fire: Implicit Routing

In [11]:
command_one = lambda: print('Routed to command_one variable!')

def command_two():
    print('Routed to command_two function!')

class CommandGroup(object):
    """Grouped commands."""
    def command_one(self):
        print('Routed to CommandGroup command_three method!')

def main():
    fire.Fire()

In [12]:
%%bash
python ./demo/routing_implicit.py command_one
python ./demo/routing_implicit.py command_two
python ./demo/routing_implicit.py CommandGroup command_one

Routed to command_one variable!
Routed to command_two function!
Routed to CommandGroup command_three method!


In [13]:
%%bash
python ./demo/overview.py __builtins__ print 'Expose explicty!'

Expose explicty!


# Harnessing Fire: Explicit Routing

In [14]:
command_one = lambda: print('Routed to command_one variable!')

def command_two():
    print('Routed to command_two function!')

class CommandGroup(object):
    def command_one(self):
        print('Routed to CommandGroup command_three method!')

def main():
    fire.Fire({
        'command_one': command_one,
        'command_two': command_two,
        'group': CommandGroup,
        'command_three': CommandGroup().command_one})

In [15]:
%%bash
python ./demo/routing_basics.py command_one
python ./demo/routing_basics.py command_two
python ./demo/routing_basics.py command_three
python ./demo/routing_basics.py group command_one

Routed to command_one variable!
Routed to command_two function!
Routed to CommandGroup command_one method!
Routed to CommandGroup command_one method!


# Harnessing Fire: Commands from Variables

- recursive calls on returned objects affords implicit chaining
- parameters of assigned functions accepted as inputs
- alias long identifers in routes passed to Fire

In [22]:
import fire
from translate import Translator

english = 'Hello IndyPy!'
german = Translator('de').translate
arabic = lambda x: Translator('ar').translate(x)

def main():
    fire.Fire({'en': english, 'de': german, 'ar': arabic})

# Harnessing Fire: Commands from Variables

In [23]:
%%bash
python ./demo/variables.py en
python ./demo/variables.py en split

Hello IndyPy!
Hello
IndyPy!


In [24]:
%%bash
python ./demo/variables.py de 'Hello IndyPy'

Hallo IndyPy


In [25]:
%%bash
python ./demo/variables.py ar 'Hello IndyPy'

مرحبا انديبي


# Harnessing Fire: Commands from Functions

- function identifiers automatically mapped to commands
- postional arguments are required
- keyword arguments are optional
- method chaining possible on returned values
- underscores in function & parameter identifiers converted to hyphens

In [26]:
import fire
from translate import Translator

GREETING = 'Hello {}, welcome to IndyPy!'

def greet(name, language='en', msg=GREETING):
    """Greet people in their preferred language."""
    return Translator(language).translate(msg.format(name))

def main():
    fire.Fire()

In [27]:
%%bash
python ./demo/functions.py greet everyone --language=zh

大家好，欢迎来到IndyPy！


# Harnessing Fire: Commands from Classes

- exposes all "public" methods and attributes
- "private" (\_, \_\_) prefixed methods and attributes not exposed
- use --option=convention for posititional & keyword arguments of initializers
- initializers' parameters may be positional or keyword arguments
- keyword arguments order independent across class methods
- underscores in method & parameter identifiers converted to hyphens

# Harnessing Fire: Commands from Classes

In [28]:
import fire
from translate import Translator

LANGUAGES = ('en', 'es')

class Greeter(object):
    """Demonstrate class use in Google's Python Fire."""
    
    def __init__(self, speaks=LANGUAGES):
        self.supported_languages = speaks

    def _is_supported_language(self, target_language: str) -> bool:
        return target_language in self.supported_languages

    def greet(self, name, salute='Hello', language='en'):
        """Greet people in their preferred language."""
        if self._is_supported_language(language):
            return Translator(language).translate('{} {}!'.format(salute, name))
        else:
            return '{} {}'.format(salute, name)

    def depart(self, name, salute='Goodbye', language='en'):
        """Bid people farewell in their preferred language."""
        return self.greet(name, salute=salute, language=language)

def main():
    fire.Fire(Greeter)

# Harnessing Fire: Commands from Classes

In [29]:
%%bash
python ./demo/classes.py greet IndyPy
python ./demo/classes.py greet IndyPy --language=es
python ./demo/classes.py  --speaks='(zh, ur)' greet IndyPy --language=ur
python ./demo/classes.py greet IndyPy --language=zh --speaks='(zh, ur)'
python ./demo/classes.py --language=zh --speaks='(zh, ur)' greet IndyPy 

Hello IndyPy!
Hola IndyPy!
ہیلو انڈیپی!
你好IndyPy！
你好IndyPy！


In [30]:
%%bash
python ./demo/classes.py depart IndyPy
python ./demo/classes.py depart IndyPy --language=es
python ./demo/classes.py  --speaks='(en, cy)' depart IndyPy --language=cy
python ./demo/classes.py depart IndyPy --language=cy --speaks='(en, cy)' 
python ./demo/classes.py --language=cy --speaks='(en, cy)' depart IndyPy 

Goodbye IndyPy!
Adiós IndyPy!
Hwyl fawr IndyPy!
Hwyl fawr IndyPy!
Hwyl fawr IndyPy!


# Harnessing Fire: Grouped & Nested Routes

In [45]:
class NestedGroup(object):

    def command(self, arg):
        print('NestedGroup received: {}'.format(arg))


class ParentGroup(object):
    nested = NestedGroup()

    def command(self, arg):
        print('ParentGroup received: {}'.format(arg))

def main():
    fire.Fire(ParentGroup)

In [43]:
%%bash
python demo/nesting.py command 'grouped!'

ParentGroup received: grouped!


In [44]:
%%bash
python demo/nesting.py nested command 'composition!'

NestedGroup received: composition!


# Harnessing Fire: Pipelining

```python
class Greeter(MultiLinguist):
    """Demonstrate class method chaining in Google's Python Fire."""

    def greet(self, name):
        """Greet persons in their native language."""
        print(self._translate('{} {}'.format(self.greets_with, name)))
        return self

    def depart(self, name):
        """Farewell persons in their native language."""
        print(self._translate('{} {}'.format(self.departs_with, name)))
        return self

    def run(self):
        """Stop Fire from displaying self returned from chained methods."""
        pass
```

In [19]:
%%bash
python ./demo/chaining.py greet IndyPy depart IndyPy run

Hello IndyPy
Goodbye IndyPy


# Harnessing Fire: Command Parameters

Pass any literal supported by the version of Python used

1. booleans
1. dictionaries
1. floats
1. integers
1. lists
1. strings
1. tuples
1. ANY literal

# Harnessing Fire: Command Parameters

## Booleans

In [31]:
%%bash
python ./demo/harnessing_fire.py parameters True
python ./demo/harnessing_fire.py parameters False

args: (True,)
args: (False,)


# Harnessing Fire: Command Parameters

## Dictionaries

In [32]:
%%bash
python ./demo/harnessing_fire.py parameters '{a: 1, b: 2, c: 3}'
python ./demo/harnessing_fire.py parameters '{a: z, b: y, c: x}'

args: ({'a': 1, 'b': 2, 'c': 3},)
args: ({'a': 'z', 'b': 'y', 'c': 'x'},)


# Harnessing Fire: Command Parameters

## Floats

In [33]:
%%bash
python ./demo/harnessing_fire.py parameters 2.
python ./demo/harnessing_fire.py parameters .2
python ./demo/harnessing_fire.py parameters 2.0

args: (2.0,)
args: (0.2,)
args: (2.0,)


# Harnessing Fire: Command Parameters

## Integers

In [34]:
%%bash

#integers
python ./demo/harnessing_fire.py parameters 1
python ./demo/harnessing_fire.py parameters '4'
python ./demo/harnessing_fire.py parameters "'4'"

args: (1,)
args: (4,)
args: ('4',)


# Harnessing Fire: Command Parameters

## Lists

In [35]:
%%bash
python ./demo/harnessing_fire.py parameters '[a, b, c]'

args: (['a', 'b', 'c'],)


# Harnessing Fire: Command Parameters

## Strings

In [36]:
%%bash
python ./demo/harnessing_fire.py parameters three
python ./demo/harnessing_fire.py parameters 'three'
python ./demo/harnessing_fire.py parameters "'three' is a magic number"
python ./demo/harnessing_fire.py parameters '"three" is a magic number'
python ./demo/harnessing_fire.py parameters '"three" is a magic number\n yes it is'

args: ('three',)
args: ('three',)
args: ("'three' is a magic number",)
args: ('"three" is a magic number',)
args: ('"three" is a magic number\\n yes it is',)


# Harnessing Fire: Command Parameters

## Tuples

In [37]:
%%bash
python ./demo/harnessing_fire.py demonstrate_parameters '(a, b, c)'
python ./demo/harnessing_fire.py demonstrate_parameters '(1, 2, 3)'

args: (('a', 'b', 'c'),)
args: ((1, 2, 3),)


# Harnessing Fire: Commands from Modules

```python
import fire
from translate import Translator

LANGUAGES = ('en', 'es')

...

if __name__ == '__main__':
    fire.Fire()
```

In [38]:
%%bash
# from translate import Translator
python ./demo/rad.py Translator --to_lang=de translate "Expose explicitly!"

Explizit ausstellen!


# Testing Fire

1. subprocess module
1. fixtures
1. test cases

# Testing Fire: subprocess Module

Spawn new processes, connect to their input/output/error pipes, and obtain their return codes.

In [39]:
import subprocess
cmd = "echo 'TDD for CLIs'"
result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE)
print(result.stdout.decode('utf-8'))

TDD for CLIs



- subprocess.run() accepts command string
- shell=True creates shell session
- stdout=subprocess.PIPE captures standard out
- decode stdout to utf-8 for assertions
- pass environment variables with env={'IDENTIFIER': 'value'} kwarg

# Testing Fire: py.test Fixtures

Testing scripts before packaging requires executing scripts with python. Use os.path and py.test fixtures for DRY path generation.
- fixtures from conftest.py available as test parameters
- use closure to complete paths as needed
- session scope flags fixture to run once

In [40]:
from os.path import abspath, dirname, join #don't do this!
import pytest

@pytest.fixture(scope='session')
def path_builder():
    
    PROJECT_ROOT = dirname(dirname(abspath(__file__)))

    def func(directory, filename):
        return join(PROJECT_ROOT, directory, filename)

    return func

# Testing Fire: Test Cases

In [41]:
import subprocess

class TestScript():
    
    script_tested = ('demo', 'rad.py')

    def test_greet_english(self, path_builder):
        script = path_builder(*self.script_tested)
        command = ' '.join(['python', script, 'greet', 'everyone'])
        result = subprocess.run(command, shell=True, stdout=subprocess.PIPE)
        expected_result = 'Hello everyone, welcome to Automate!\n'
        assert result.stdout.decode('utf-8') == expected_result

    def test_greet_hindi(self, path_builder):
        script = path_builder(*self.script_tested)
        command = ' '.join(['python', script, 'greet', 'everyone', '--lang=hi'])
        result = subprocess.run(command, shell=True, stdout=subprocess.PIPE)
        expected_result = 'सभी को नमस्कार, स्वचालित में आपका स्वागत है!\n'
        assert result.stdout.decode('utf-8') == expected_result

# Testing Fire

In [42]:
%%bash
pytest

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-4.0.2, py-1.7.0, pluggy-0.8.0
rootdir: /Users/dash/my.presentations/google_fire, inifile:
plugins: remotedata-0.3.1, openfiles-0.3.1, doctestplus-0.2.0, arraydiff-0.3
collected 2 items

tests/test_rad.py ..                                                     [100%]

=========================== 2 passed in 1.42 seconds ===========================


# Packaging Fire: cookiecutter-pypackage

__cookiecutter__ automates project boilerplate with the Jinja2 templating engine and git.

__cookiecutter-pypackage__ is a boilerplate template for scaffolding python projects.

```bash
pip install -U cookiecutter
cd ~/some/directory
cookiecutter https://github.com/audreyr/cookiecutter-pypackage
```

# Packaging Fire: cookiecutter-pypackage

- full_name, email, github_username
- project_name [Python Boilerplate]
- project_slug [indypy_packaging_demo]
- project_short_description [Python Boilerplate contains ...]
- pypi_username [infosmith]
- version [0.1.0]
- use_pytest [n]
- use_pypi_deployment_with_travis [y]

# Packaging Fire: Entry Points

Entry points defined in setup.py are how setuptools make your scripts available to the console.

```python
entry_points={
    'console_scripts': [
        'indypy = fire_starter.fire_starter:main',
    ],
}
```

```bash
pip install . -e
```

# Packaging Fire: Entry Points

```python
#!/Users/dash/.local/share/virtualenvs/fire_starter-RZ18EKnQ/bin/python3
# EASY-INSTALL-ENTRY-SCRIPT: 'fire-starter','console_scripts','indypy'
__requires__ = 'fire-starter'
import re
import sys
from pkg_resources import load_entry_point

if __name__ == '__main__':
    sys.argv[0] = re.sub(r'(-script\.pyw?|\.exe)?$', '', sys.argv[0])
    sys.exit(
        load_entry_point('fire-starter', 'console_scripts', 'indypy')()
    )
```

# Distributing Fire Scripts on PyPi

## Manually

```bash
#test PyPI upload
python setup.py sdist bdist_wheel
twine upload --repository-url https://test.pypi.org/legacy/ dist/*

#push to PyPI
twine upload dist/*
```

## cookiecutter-pypackages MakeFile

```make
release: dist ## package and upload a release
	twine upload dist/*
```

# Fire Prevention

1. be explicit about what is exposed
1. avoid naming functions/methods/parameters for typing convenience
1. nesting not exactly simplified
1. identifiers promoted to global scope receive a CLI
1. Fire's help menu is developer oriented

# Examples of Fire Alternatives

1. argv
2. argparse
3. docopt
4. Click
5. Cement

# argv

```python
import sys

def greet(name):
    return 'Hello {}!'.format(name)

def depart(name):
    return 'Goodbye {}!'.format(name)

def is_valid_arguments(arguments):
    return (2 < len(arguments) < 4) and arguments[1] in ['greet', 'depart']

def main(args):
    if not is_valid_arguments(args):
        print('Unhandled arguments: {}'.format(sys.argv))
    elif args[1] == 'greet':
        print(greet(sys.argv[2]))
    elif args[1] == 'depart':
        print(depart(sys.argv[2]))

if __name__ == '__main__':
    main(sys.argv)
```

# argparse

```python
import argparse

parser = argparse.ArgumentParser()

parser.add_argument('command', help='Say greeting or farewell.')
parser.add_argument('name', help='Name to salute.')

def greet(name):
    return 'Hello {}!'.format(name)

def depart(name):
    return 'Goodbye {}!'.format(name)

def main(args):
    if args.command == 'greet':
        print(greet(args.name))
    elif args.command == 'depart':
        print(depart(args.name))
    else:
        print('Unhandled arguments: {}'.format(args))

if __name__ == '__main__':
    main(parser.parse_args())
```

# docopt

```python
"""
Scripting with docopt.

Usage:
    hello_docopt.py greet <greet_name>
    hello_docopt.py depart <depart_name>

Options:
    -h --help     Show this screen.
"""

def greet(name):
    return 'Hello {}!'.format(name)

def depart(name):
    return 'Goodbye {}!'.format(name)

def main(args):
    if args['greet']:
        print(greet(args['<greet_name>']))
    elif args['depart']:
        print(depart(args['<depart_name>']))
    else:
        print('Unhandled arguments: {}'.format(args))

if __name__ == '__main__':
    from docopt import docopt

    script_arguments = docopt(__doc__)
    main(script_arguments)
```

# Click

```python
import click


@click.group()
def cli():
    pass


@cli.command()
@click.argument('name')
def greet(name):
    """Echo greeting."""
    click.echo('Hello {}!'.format(name))


@cli.command()
@click.argument('name')
def depart(name):
    """Echo farewell."""
    click.echo('Goodbye {}!'.format(name))


cli.add_command(greet)
cli.add_command(depart)

if __name__ == '__main__':
    cli()
```

# Cement

<img src="./images/killing_mosquitos.jpg" />

# Conclusions

<img src='./images/standards.png' />